# Setup 

In [1]:
!pip install -q matplotlib

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

print("TensorFlow version: {}".format(tf.__version__))
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

TensorFlow version: 2.8.0


# Load the data in memory

For any small CSV vitals the simplest way to train a TensorFlow model on it is to load it into memory as a pandas Dataframe or a NumPy array.

## Pandas

In [ ]:
# using Pandas
# read in only the columns for exploratory prediction
#vitals = pd.read_csv('../data/raw/pat_vitals_labeled-dataSepsis.csv', usecols=['HR','Temp','Resp','isSepsis'])
# print the data
#vitals.head()

## TFData

In [19]:
# using TFData
vitals_file_path = '../data/raw/pat_vitals_labeled-dataSepsis.csv'
# reads CSV files into a dataset
vitals_csv_ds = tf.data.experimental.make_csv_dataset(
    vitals_file_path,
    batch_size=5, # Artificially small to make examples easier to show.
    label_name='isSepsis', # data for this column is returned as a separate Tensor from the features dictionary
    num_epochs=1, # int specifying the number of times this dataset is repeated
    shuffle=True, #  bool that indicates whether the input should be shuffled
    shuffle_buffer_size=100000,
    shuffle_seed=42, # randomization seed to use for shuffling
    ignore_errors=True)

In [20]:
for batch, label in vitals_csv_ds.take(1):
  for key, value in batch.items():
    print(f"{key:20s}: {value}")
  print()
  print(f"{'label':20s}: {label}")

patient_id          : [10884  3576 14519  2020  3352]
record_date         : [b'' b'' b'' b'' b'']
record_time         : [b'' b'' b'' b'' b'']
HR                  : [88 60 86 79 71]
O2Sat               : [b'96' b'100' b'94' b'96' b'100']
Temp                : [35.9 35.8 36.2 42.  35.3]
SBP                 : [b'95' b'96' b'97' b'143' b'124']
MAP                 : [b'67' b'73.33' b'57' b'105' b'73']
DBP                 : [b'57' b'NaN' b'42' b'81' b'53']
Resp                : [27 16 15 18 18]
EtCO2               : [b'NaN' b'NaN' b'NaN' b'NaN' b'NaN']

label               : [1 0 0 0 0]


In [2]:
# look at just a few items from the dataset
for item in vitals_csv_ds.take(1):
    print(item)

NameError: name 'vitals_csv_ds' is not defined

# Preprocess

The nominal task for this vitals is to predict if the patient is septic from the other measurements, so separate the features and labels for training:

## Pandas

In [25]:
#vitals_features = vitals_csv_ds.copy()
#vitals_labels = vitals_features.pop('isSepsis')

For this vitals you will treat all features identically. Pack the features into a single NumPy array.:

In [26]:
#vitals_features = np.array(vitals_features)
#vitals_features

It's good practice to normalize the inputs to your model. The Keras preprocessing layers provide a convenient way to build this normalization into your model. Note: Only use your training data to .adapt() preprocessing layers. Do not use your validation or test data.

The layer will precompute the mean and variance of each column, and use these to normalize the data.

In [27]:
#normalize = layers.Normalization()

normalize.adapt(abalone_features)
Then you use the Normalization.adapt() method to adapt the normalization layer to your data.

## TF Data

# Develop

Next make a regression model predict the age. Since there is only a single input tensor, a keras.Sequential model is sufficient here.

In [ ]:
norm_sepsis_model = tf.keras.Sequential([
  normalize,
  layers.Dense(64),
  layers.Dense(1)
])

norm_sepsis_model.compile(loss = tf.losses.MeanSquaredError(),
                           optimizer = tf.optimizers.Adam())

# Train

To train that model, pass the features and labels to Model.fit:

In [ ]:
norm_sepsis_model.fit(vitals_features, vitals_labels, epochs=10)

In [ ]:
predictions = norm_sepsis_model(vitals_features[:1].numpy()
predictions

# Evaluate